In [4]:
import os
import sys
import json
import time
import random
from sklearn.model_selection import train_test_split

In [29]:
import torch
import numpy as np

In [25]:
a = torch.FloatTensor([0.0 if i != 0 else 1.0 for i in range(2)])

In [40]:
a = [1,1]

In [41]:
a.append(2) if a[0] == 0 else a.append(4)

In [42]:
a

[1, 1, 4]

In [37]:
(a[1] ==1) == False

False

In [2]:
path = './data/NCI_txt/NCI-H23/'

In [3]:
edges_raw = open('%sNCI-H23_A.txt'%path).read().split('\n')[:-1]
graph_ID_of_nodes = open('%sNCI-H23_graph_indicator.txt'%path).read().split('\n')[:-1]
node_labels_raw = open('%sNCI-H23_node_labels.txt'%path).read().split('\n')[:-1]
graph_labels_raw = open('%sNCI-H23_graph_labels.txt'%path).read().split('\n')[:-1]

In [5]:
#####
begin = time.time()
track = 0
start_node = 0
start_edge = 0
test = 0
graphs = []
#####
for i in graph_labels_raw :
    ###### Add graph target
    data = {}
    data['target'] = int(i)

    ######
    try:
        num_nodes = graph_ID_of_nodes.index(str(track+2)) - start_node
    except:
        num_nodes = len(graph_ID_of_nodes) - int(graph_ID_of_nodes.index(str(track+1)))
    
    nodes_labels = node_labels_raw[start_node:num_nodes+start_node]
    
    
    ###### Add node labels
    data['labels'] = {'0':nodes_labels[0]}
    nodes_identifier = 1
    for label in nodes_labels[1:]:
        data['labels'][str(nodes_identifier)] = label
        nodes_identifier+=1
       
    ###### Add edges
    edges = []
    
    for pair in edges_raw[start_edge:]:
        last = int(pair.split(',')[-1])
        first = int(pair.split(',')[0])

        if last <= (start_node+num_nodes):
            first = first - start_node -1
            last = last - start_node -1
            edges.append([first,last])
        else:
            break
            
    data['edges'] = edges
    
    graphs.append(data)
    ####### update tracking
    start_node+=num_nodes
    start_edge+=len(edges)
    track+=1
    ######
    if track % 1000 == 0:
        print(track)


print('Time: %s'%(time.time()-begin))
    

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
Time: 1397.3600690364838


In [8]:
sample = []
count = 0
track = 0
X = []
y = []

ID = [i for i in range(len(graphs))]
random.shuffle(ID)

for i in ID:
    if graphs[i]['target'] == 1:
        sample.append(graphs[i])
        X.append(track)
        y.append(graphs[i]['target'])
        track+=1
        
for i in ID:
    if graphs[i]['target'] == 0:
        sample.append(graphs[i])
        X.append(track)
        y.append(graphs[i]['target'])
        track+=1
        count+= 1
    if count > 2100 :
        break

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [20]:
####### Save file
track = 0
for i in X_train:
    outdir = './input/train/'
    outfile = open(os.path.join(outdir,os.path.basename('graph_%s.json'%track)),'w')
    outfile.write(json.dumps(sample[i]))  
    outfile.close()
    ########
    track+=1
    ######

for i in X_test:
    outdir = './input/test/'
    outfile = open(os.path.join(outdir,os.path.basename('graph_%s.json'%track)),'w')
    outfile.write(json.dumps(sample[i]))  
    outfile.close()
    ########
    track+=1
    ######